[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/satyaki-mallick/DeepLearningAssignment1/blob/master/Assignment_1.ipynb#scrollTo=7uut_aem5B5q)

# Assignment 1

<b>Group [fill in group number]</b>
* <b> Student 1 </b> : SATYAKI MALLICK + 1410881
* <b> Student 2 </b> : HUILIN + Huilin_student_no

**Reading material**
* [1] Mikolov, Tomas, et al. "[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)" Advances in neural information processing systems. 2013. 

<b><font color='red'>NOTE</font></b> When submitting your notebook, please make sure that the training history of your model is visible in the output. This means that you should **NOT** clean your output cells of the notebook. Make sure that your notebook runs without errors in linear order.



# Question 1 - Keras implementation (10 pt)

### Word embeddings
Build word embeddings with a Keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training. Use a window size of 2 to train the embeddings (`window_size` in the jupyter notebook). 

1. Build word embeddings of length 50, 150 and 300 using the Skipgram model
2. Build word embeddings of length 50, 150 and 300 using CBOW model
3. Analyze the different word embeddings:
    - Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in the paper. Do not use existing libraries for this task such as Gensim. 
Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. $e_{x}$ denotes the embedding of word $x$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.
    - Give at least 5 different  examples of analogies.
    - Compare the performance on the analogy tasks between the word embeddings and briefly discuss your results.

4. Discuss:
  - Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


<b>HINT</b> See practical 3.1 for some helpful code to start this assignment.


### Import libraries

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import keras.backend as K
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence

# other helpful libraries
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
import pandas as pd
import pickle

Using TensorFlow backend.


In [0]:
print(tf.__version__) #  check what version of TF is imported

2.2.0


### Import file

If you use Google Colab, you need to mount your Google Drive to the notebook when you want to use files that are located in your Google Drive. Paste the authorization code, from the new tab page that opens automatically when running the cell, in the cell below.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Navigate to the folder in which `alice.txt` is located. Make sure to start path with '/content/drive/My Drive/' if you want to load the file from your Google Drive.

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/'

/content/drive/My Drive/Colab Notebooks


In [0]:
file_name = 'alice.txt'
corpus = open(file_name).readlines()

### Data preprocessing

See Practical 3.1 for an explanation of the preprocessing steps done below.

In [0]:
# Removes sentences with fewer than 3 words
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

# remove punctuation in text and fit tokenizer on entire corpus
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)

# convert text to sequence of integer values
corpus = tokenizer.texts_to_sequences(corpus)
n_samples = sum(len(s) for s in corpus) # total number of words in the corpus
V = len(tokenizer.word_index) + 1 # total number of unique words in the corpus

In [0]:
n_samples, V

(27165, 2557)

In [0]:
# example of how word to integer mapping looks like in the tokenizer
print(list((tokenizer.word_index.items()))[:5])

[('the', 1), ('and', 2), ('to', 3), ('a', 4), ('it', 5)]


In [0]:
# parameters
window_size = 2
window_size_corpus = 4

In [0]:
def save_embeddings(model, model_name):
  weights = model.get_weights()
  embedding = weights[0]

  path= './embedding_{}.pth'.format(model_name)
  pickle.dump(embedding, open(path,'wb'))

  f = open(model_name + '.txt', 'w')
  f.write(" ".join([str(V-1), str(dim)]))
  f.write("\n")

  for word,i in tokenizer.word_index.items():
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(embedding[i,:]))))
    f.write("\n")
  f.close()

In [0]:
# To load the pickled embedding
def unpickle(model_name):
  path = './embedding_{}.pth'.format(model_name)
  unpickled_embedding = pickle.load(open(path,'rb'))
  open(path,'rb').close()
  return unpicked_embedding

## Task 1.1 - Skipgram
Build word embeddings of length 50, 150 and 300 using the Skipgram model.

In [0]:
#prepare data for skipgram
def generate_data_skipgram(corpus, window_size, V):
    # TODO Implement here
    # HINT: see Practical 3.1
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    # Add the input word
                    #in_words.append(word)
                    all_in.append(word)
                    # Add one-hot of the context words
                    all_out.append(to_categorical(words[i], V))
                                      
    return (np.array(all_in),np.array(all_out))

In [0]:
# create training data
x_skipgram , y_skipgram = generate_data_skipgram(corpus,window_size,V)

In [0]:
# x each word represented as a number and then a window created

In [0]:
# Sample of y:
# rows represent each word.
# columns represent total unique words
# sample array for y for input [1, 0, 3, 4, 5, 0, 2, 1]
# array([[ 0.,  1.,  0.,  0.,  0.,  0.],
#        [ 1.,  0.,  0.,  0.,  0.,  0.],
#        [ 0.,  0.,  0.,  1.,  0.,  0.],
#        [ 0.,  0.,  0.,  0.,  1.,  0.],
#        [ 0.,  0.,  0.,  0.,  0.,  1.],
#        [ 1.,  0.,  0.,  0.,  0.,  0.],
#        [ 0.,  0.,  1.,  0.,  0.,  0.],
#        [ 0.,  1.,  0.,  0.,  0.,  0.]])

In [0]:
x_skipgram.shape, y_skipgram.shape

((94556,), (94556, 2557))

In [0]:
V = y_skipgram.shape[1]

In [0]:
def skipgram_architechture(dim):
  model = Sequential()
  model.add(Embedding(input_dim=V, output_dim=dim, input_length=1, embeddings_initializer='glorot_uniform'))
  # not sure about the input length
  #model.add(Reshape((94556,), input_shape=(1,94556)))
  # above line or below line
  model.add(Reshape((dim,)))
  model.add(Dense(V, activation='softmax', kernel_initializer='glorot_uniform'))
  model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return model

### Skipgram for Embedding Vector Length 50

In [0]:
dim = 50
skipgram50 = skipgram_architechture(dim)
skipgram50.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1, 50)             127850    
_________________________________________________________________
reshape_9 (Reshape)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2557)              130407    
Total params: 258,257
Trainable params: 258,257
Non-trainable params: 0
_________________________________________________________________


<b>HINT</b>: To increase training speed of your model, you can use the free available GPU power in Google Colab. Go to `Edit` --> `Notebook Settings` --> select `GPU` under `hardware accelerator`.

In [0]:
# train skipgram model
skipgram50.fit(x_skipgram, y_skipgram, batch_size=64, epochs=10)
save_embeddings(skipgram50, 'skipgram_50')

Epoch 1/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8462 - accuracy: 3.8073e-04
Epoch 2/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8454 - accuracy: 6.1339e-04
Epoch 3/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8445 - accuracy: 0.0012
Epoch 4/10
1478/1478 [==============================] - 7s 4ms/step - loss: 7.8437 - accuracy: 0.0030
Epoch 5/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8429 - accuracy: 0.0084
Epoch 6/10
1478/1478 [==============================] - 6s 4ms/step - loss: 7.8420 - accuracy: 0.0149
Epoch 7/10
1478/1478 [==============================] - 7s 5ms/step - loss: 7.8412 - accuracy: 0.0219
Epoch 8/10
1478/1478 [==============================] - 7s 5ms/step - loss: 7.8404 - accuracy: 0.0276
Epoch 9/10
1478/1478 [==============================] - 7s 4ms/step - loss: 7.8395 - accuracy: 0.0333
Epoch 10/10
1478/1478 [==============================] - 6s 4ms/step - los

### Skipgram for embedding vector length 150

In [0]:
# save embeddings for vectors of length 50, 150 and 300 using skipgram model
dim = 150
skipgram150 = skipgram_architechture(dim)
skipgram150.summary()
skipgram150.fit(x_skipgram, y_skipgram, batch_size=64, epochs=10)
#save_embeddings(skipgram150, 'skipgram_150')


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1, 150)            383550    
_________________________________________________________________
reshape_11 (Reshape)         (None, 150)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 2557)              386107    
Total params: 769,657
Trainable params: 769,657
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1478/1478 [==============================] - 9s 6ms/step - loss: 7.8459 - accuracy: 5.0764e-04
Epoch 2/10
1478/1478 [==============================] - 9s 6ms/step - loss: 7.8450 - accuracy: 0.0024
Epoch 3/10
1478/1478 [==============================] - 9s 6ms/step - loss: 7.8442 - accuracy: 0.0038
Epoch 4/10
1478/1478 [==============================] - 9s 6ms/step

### Skipgram for embedding vector length 300

In [0]:
dim = 300
skipgram300 = skipgram_architechture(dim)
skipgram300.fit(x_skipgram, y_skipgram, batch_size=32, epochs=10)
save_embeddings(skipgram300, 'skipgram_300')

Epoch 1/10
2955/2955 [==============================] - 19s 6ms/step - loss: 7.8458 - accuracy: 4.6533e-04
Epoch 2/10
2955/2955 [==============================] - 18s 6ms/step - loss: 7.8443 - accuracy: 0.0011
Epoch 3/10
2955/2955 [==============================] - 18s 6ms/step - loss: 7.8428 - accuracy: 0.0017
Epoch 4/10
2955/2955 [==============================] - 18s 6ms/step - loss: 7.8414 - accuracy: 0.0050
Epoch 5/10
2955/2955 [==============================] - 18s 6ms/step - loss: 7.8399 - accuracy: 0.0095
Epoch 6/10
2955/2955 [==============================] - 18s 6ms/step - loss: 7.8384 - accuracy: 0.0132
Epoch 7/10
2955/2955 [==============================] - 19s 6ms/step - loss: 7.8369 - accuracy: 0.0166
Epoch 8/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8355 - accuracy: 0.0228
Epoch 9/10
2955/2955 [==============================] - 20s 7ms/step - loss: 7.8340 - accuracy: 0.0272
Epoch 10/10
2955/2955 [==============================] - 19s 6ms/step

## Task 1.2 - CBOW

Build word embeddings of length 50, 150 and 300 using CBOW model.

In [0]:
# prepare data for CBOW

# create training data

# create CBOW architecture

# train CBOW model

# save embeddings for vectors of length 50, 150 and 300 using CBOW model

Prepare data for CBOW



In [0]:
def generate_data_CBOW(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = [] #one real label wt
    for line in corpus:
        sentence_length = len(line)
      
        for index, word in enumerate(line):  #for each word in the line, we create a little neighborhood [left,right]
            left = index - window_size
            right = index + window_size + 1
      
            in_words = []   #neighbor words of wt, used as input to predict wt       
      
            for i in range(left, right):
                if 0 <= i < sentence_length and i != index:
                    # Add the input word
                    in_words.append(line[i])

            
            all_in.append(in_words)
            all_out.append(to_categorical(word,V))
      
    all_in = sequence.pad_sequences(all_in, maxlen=maxlen)
                                      
    return (np.array(all_in),np.array(all_out))

Create training data

In [0]:
x , y = generate_data_CBOW(corpus,window_size,V)

Create CBOW architecture

In [0]:
def cbow_architechture(dim):
    cbow = Sequential()
    cbow.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=window_size*2))
    cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
    cbow.add(Dense(V, kernel_initializer='glorot_uniform', activation='softmax'))
    #multiclass classification->categorical_crossentropy loss, optimizer->PPT02 p24
    cbow.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return cbow

###CBOW for Embedding Vector Length 50

Train CBOW model - embedding vector length 50

In [0]:
# dimension of word embedding
dim = 50

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 4, 50)             127850    
_________________________________________________________________
lambda_1 (Lambda)            (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2557)              130407    
Total params: 258,257
Trainable params: 258,257
Non-trainable params: 0
_________________________________________________________________


In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8463 - accuracy: 0.0012
Epoch 2/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8461 - accuracy: 0.0015
Epoch 3/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8459 - accuracy: 0.0021
Epoch 4/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8456 - accuracy: 0.0035
Epoch 5/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8454 - accuracy: 0.0049
Epoch 6/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8451 - accuracy: 0.0064
Epoch 7/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8449 - accuracy: 0.0089
Epoch 8/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8447 - accuracy: 0.0116
Epoch 9/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8444 - accuracy: 0.0142
Epoch 10/10
425/425 [==============================] - 2s 4ms/step - loss: 7.8442 - accuracy: 0.0174

Save embedding

In [0]:
save_embeddings(cbow, 'vectors_cbow{dim}'.format(dim=dim))

###CBOW for Embedding Vector Length 150

Train CBOW model - embedding vector length 150

In [0]:
# dimension of word embedding
dim = 150

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Save embedding

In [0]:
save_embeddings(cbow, 'vectors_cbow{dim}'.format(dim=dim))

###CBOW for Embedding Vector Length 300

Train CBOW model - embedding vector length 300

In [0]:
# dimension of word embedding
dim = 300

cbow = cbow_architechture(dim)

In [0]:
plot_model(cbow, show_shapes = True, show_layer_names=False)

In [0]:
cbow.summary()

In [0]:
# train skipgram model
cbow.fit(x, y, batch_size=64, epochs=10, verbose=1)

Save embedding

In [0]:
save_embeddings(cbow, 'vectors_cbow{dim}'.format(dim=dim))

## Task 1.3 - Analogy function

Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in [1]. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. 

In a perfect scenario, we would like that this analogy ( $e_{king} - e_{queen} + e_{woman}$) results in the embedding of the word "man". However, it does not always result in exactly the same word embedding. The result of the formula is called the expected or the predicted word embedding. In this context, "man" is called the true or the actual word $t$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ ($e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.  

You have to answer an analogy function using each embedding for both CBOW and Skipgram model. This means that for each analogy we have 6 outputs. Show the true word (with distance similarity value between predicted embedding and true word embedding, i.e. `sim1`) , the predicted word (with distance similarity value between predicted embedding and the embedding of the word in the vocabulary that is closest to this predicted embedding, i.e. `sim2`) and a boolean answer whether the predicted word **exactly** equals the true word. 

<b>HINT</b>: to visualize the results of the analogy tasks , you can print them in a table. An example is given below.


| Analogy task | True word (sim1)  | Predicted word (sim2) | Embedding | Correct?|
|------|------|------|------|------|
|  queen is to king as woman is to ?	 | man (sim1) | predictd_word(sim2) | SG_50 | True / False|

* Give at least 5 different  examples of analogies.
* Compare the performance on the analogy s between the word embeddings and briefly discuss your results.

In [0]:
def get_embedding_from_file(name):
  file = open(name + '.txt', 'r')
  return file

In [0]:
embedding = get_embedding_from_file('skipgram_50')
embed('king', embedding)

TypeError: ignored

In [0]:
def embed(word, embedding, vocab_size=V, tokenizer=tokenizer):
  int_word = tokenizer.texts_to_sequences([word])[0]
  bin_word = to_categorical(int_word, V)
  return np.dot(bin_word, embedding)

In [0]:
embedding = get_embedding_from_file(model_name)
predicted_embedding = embed('king', embedding) - embed('queen', embedding) + embed('woman', embedding) 

In [0]:
print(distance_exp_true = np.linalg.norm(predictedEmbedding - embed('man')))

## Task 1.4 - Discussion
Answer the following question:
* Given the same number of sentences as input, CBOW and Skipgram arrange the data into different number of training samples. Which one has more and why?


# Question 2 - Peer review (0 pt):
Finally, each group member must write a single paragraph outlining their opinion on the work distribution within the group. Did every group member
contribute equally? Did you split up tasks in a fair manner, or jointly worked through the exercises. Do you think that some members of your group deserve a different grade from others? You can use the table below to make an overview of how the tasks were divided:



| Student name | Task  |
|------|------|
|  student name 1  | task x |
| student name 2  | task x|
| everyone | task x|


Doubts

-- What is V in CBOW model?

-- How to read the embeddings again from the files for Q 1.3?